# Specter Embeddings
Delvin

This notebook
1) preps SickKids IPE + Urology Datasets for embedding
2) performs inference to embed the abstracts

Output is a .jsonl file to be parsed.


Follow instructions for setting up specter per: https://github.com/allenai/specter

Then, download datasets_complete and unzip into `specter`

In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os
import json

In [2]:
data_dir = 'datasets_complete/*'
out_dir = 'datasets_json'

if not os.path.exists(out_dir): os.makedirs(out_dir)

In [3]:
f = glob('datasets_complete/*')

In [4]:
for fn in f: 
    basen = os.path.basename(fn)
    txt = os.path.splitext(basen)[0]
    txt = txt.split('_')[0].replace(' ', '_')

    print(txt)


Vitamin_D
Hydronephrosis
Scaling
Rehab
NCDs
ADIPP
Fragility_of_BBD
WASH


In [5]:
for fn in f:
    # construct new filename for json and ids
    print(fn)
    basen = os.path.basename(fn)
    txt = os.path.splitext(basen)[0]
    txt = txt.split('_')[0].replace(' ', '_')
    out_json = os.path.join(out_dir, '{}.json'.format(txt))
    out_ids = os.path.join(out_dir, '{}.ids'.format(txt))
    print(out_json)
    
    # read in each file and dump to json per specter formatting 
    
    dat = pd.read_csv(fn, encoding = 'ISO-8859-1')
    dat = dat.dropna(subset = ['Abstract']) # drop the papers with null abstracts after parsing to retain the index?
    dat.columns = map(str.lower, dat.columns) # lowercase for compatability w/ specter
    dat = dat.reset_index() 
    dat = dat.rename(columns={'index': 'paper_id'})
    new_json = dat[['title', 'abstract', 'paper_id']].to_json(orient = 'index')
    parsed_json = json.loads(new_json)
    #json.dumps(parsed_json, indent = 2)
    with open(out_json, 'w', encoding='utf-8') as f:
        json.dump(parsed_json, f, ensure_ascii=False, indent=4)
        
    # also create the '*.ids' file
    
    dat.to_csv(out_ids, columns=[], header=False)
    
    
print('Done!')

datasets_complete/Vitamin D_complete2.csv
datasets_json/Vitamin_D.json
datasets_complete/Hydronephrosis_FQ_complete.csv
datasets_json/Hydronephrosis.json
datasets_complete/Scaling_complete.csv
datasets_json/Scaling.json
datasets_complete/Rehab_complete.csv
datasets_json/Rehab.json
datasets_complete/NCDs_complete_2020_07_15.csv
datasets_json/NCDs.json
datasets_complete/ADIPP_complete.csv
datasets_json/ADIPP.json


/home/dso/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


datasets_complete/Fragility of BBD_complete.csv
datasets_json/Fragility_of_BBD.json


/home/dso/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


datasets_complete/WASH_complete.csv
datasets_json/WASH.json
Done!


In [ ]:
%%bash

out_dir=embed_json
mkdir -p ${out_dir}


for f in $(ls datasets_json/*ids) |
do
    f=$(basename ${f} | sed 's/\.ids//g')
    echo ${f}
    
    
    python3 scripts/embed.py \
        --ids datasets_json/${f}.ids --metadata datasets_json/${f}.json \
        --model ./model.tar.gz \
        --output-file ${out_dir}/${f}.jsonl \
        --vocab-dir data/vocab/ \
        --batch-size 96 \
        --cuda-device 2

done
echo "Done!"